In [1]:
import heapq
import pandas as pd
import random



In [2]:
data = pd.read_csv("Road_Distance.csv")
data.head()


,Distance in Kilometres,Ahmedabad,Bangalore,Bhubaneshwar,Bombay,Calcutta,Chandigarh,Cochin,Delhi,Hyderabad,...,Jaipur,Kanpur,Lucknow,Madras,Nagpur,Nasik,Panjim,Patna,Pondicherry,Pune
0,Agartala,3305,3824,2286,3593,1863,2998,4304,2708,3330,...,2801,2281,2252,3493,2696,3365,3507,1681,3661,3442
1,Agra,878,1848,1578,1202,1300,448,2278,200,1246,...,230,290,369,2048,770,1005,1715,885,2210,1214
2,Ahmedabad,-,1490,1697,552,2068,1157,1845,911,1436,...,648,1168,1247,1821,965,504,1165,1656,1818,664
3,Allahabad,1251,1686,1090,1457,817,912,2216,650,1084,...,713,193,234,2011,608,1155,1419,402,1077,1364
4,Amritsar,1356,2496,2224,1849,1919,239,3163,445,1892,...,706,926,939,2688,1416,1665,2237,1531,2856,1862


In [3]:
#construncting the graph
column_names = data.columns.tolist()
distance_values = data["Distance in Kilometres"].tolist()
column_names.pop(0)

'Distance in Kilometres'

In [4]:
cities = set(distance_values+column_names)
city_index_mapping = {}
index_city_mapping = {}
index = 0
for i in cities:
    city_index_mapping[i] = index
    index_city_mapping[index] = i
    index+=1
print(city_index_mapping)

{'Vijayawada': 0, 'Delhi': 1, 'Jamshedpur': 2, 'Hubli': 3, 'Ludhiana': 4, 'Surat': 5, 'Cochin': 6, 'Vishakapatnam': 7, 'Indore': 8, 'Jabalpur': 9, 'Nagpur': 10, 'Ahmedabad': 11, 'Nasik': 12, 'Shimla': 13, 'Varanasi': 14, 'Asansol': 15, 'Kolhapur': 16, 'Lucknow': 17, 'Baroda': 18, 'Madras': 19, 'Meerut': 20, 'Bangalore': 21, 'Bombay': 22, 'Calicut': 23, 'Panjim': 24, 'Bhubaneshwar': 25, 'Amritsar': 26, 'Agartala': 27, 'Bhopal': 28, 'Kanpur': 29, 'Ranchi': 30, 'Hyderabad': 31, 'Pune': 32, 'Patna': 33, 'Imphal': 34, 'Jullundur': 35, 'Madurai': 36, 'Jaipur': 37, 'Pondicherry': 38, 'Shillong': 39, 'Chandigarh': 40, 'Trivandrum': 41, 'Calcutta': 42, 'Coimbatore': 43, 'Gwalior': 44, 'Allahabad': 45, 'Agra': 46}


In [5]:
city_coordinates = {
    "Agartala": (23.8315, 91.2868),
    "Agra": (27.1767, 78.0081),
    "Ahmedabad": (23.0225, 72.5714),
    "Allahabad": (25.4358, 81.8463),
    "Amritsar": (31.6340, 74.8723),
    "Asansol": (23.6889, 86.9661),
    "Bangalore": (12.9716, 77.5946),
    "Baroda": (22.3072, 73.1812),
    "Bhopal": (23.2599, 77.4126),
    "Bhubaneshwar": (20.2961, 85.8245),
    "Bombay": (19.0760, 72.8777),
    "Calcutta": (22.5726, 88.3639),
    "Calicut": (11.2588, 75.7804),
    "Chandigarh": (30.7333, 76.7794),
    "Cochin": (9.9312, 76.2673),
    "Coimbatore": (11.0168, 76.9558),
    "Delhi": (28.7041, 77.1025),
    "Gwalior": (26.2183, 78.1828),
    "Hubli": (15.3647, 75.1240),
    "Hyderabad": (17.3850, 78.4867),
    "Imphal": (24.8170, 93.9368),
    "Indore": (22.7196, 75.8577),
    "Jabalpur": (23.1815, 79.9864),
    "Jaipur": (26.9124, 75.7873),
    "Jamshedpur": (22.8065, 86.2029),
    "Jullundur": (31.3260, 75.5762),
    "Kanpur": (26.4499, 80.3319),
    "Kolhapur": (16.7050, 74.2433),
    "Lucknow": (26.8467, 80.9462),
    "Ludhiana": (30.9010, 75.8573),
    "Madras": (13.0827, 80.2707),
    "Madurai": (9.9252, 78.1198),
    "Meerut": (28.9845, 77.7064),
    "Nagpur": (21.1458, 79.0882),
    "Nasik": (19.9975, 73.7898),
    "Panjim": (15.4909, 73.8278),
    "Patna": (25.5941, 85.1376),
    "Pondicherry": (11.9416, 79.8083),
    "Pune": (18.5204, 73.8567),
    "Ranchi": (23.3441, 85.3096),
    "Shillong": (25.5788, 91.8933),
    "Shimla": (31.1048, 77.1734),
    "Surat": (21.1702, 72.8311),
    "Trivandrum": (8.5241, 76.9366),
    "Varanasi": (25.3176, 82.9739),
    "Vijayawada": (16.5062, 80.6480),
    "Vishakapatnam": (17.6868, 83.2185)
}


In [6]:
#graph creation
adj = [[] for i in range(len(cities))]
for _, row in data.iterrows():
    city_a = row["Distance in Kilometres"]
    for city_b in column_names:
        if(city_a != city_b):
            adj[city_index_mapping[city_a]].append((city_index_mapping[city_b], float(row[city_b])))
            adj[city_index_mapping[city_b]].append((city_index_mapping[city_a], float(row[city_b])))
print(len(adj[41]))


20


In [8]:
def uniform_cost(n, start, goal, adj):
    frontier = [(0, start)] #(dist, city)

    dist = [float(1e9)] * (n)
    parent = list(range(n)) #to store path
    dist[start] = 0

    while frontier:
        dis, city = heapq.heappop(frontier)
        # print(city, frontier)
        for adj_city, road_distance in adj[city]:
            if dis + road_distance < dist[adj_city]:
                dist[adj_city] = dis + road_distance
                heapq.heappush(frontier, (dis + road_distance, adj_city))
                parent[adj_city] = city
    path = []
    city = goal
    while parent[city] != city:
        path.append(index_city_mapping[city])
        city = parent[city]
    path.append(index_city_mapping[start])
    path.reverse()
    return path, dist[goal]

In [9]:
def a_heuristic(city, goal):
    return ((city_coordinates[index_city_mapping[goal]][0] - city_coordinates[index_city_mapping[city]][0])**2 + (city_coordinates[index_city_mapping[goal]][1] - city_coordinates[index_city_mapping[city]][1])**2)**0.5

def i_heuristic(city, goal):
    maxi = 0
    for adj_city, road_distance in adj[city]:
        maxi = max(road_distance, maxi)
   
    return maxi
    

def astar_search(n, start, goal, adj, heuristic):
    frontier = [(0 + heuristic(start, goal), start)] #(dist, city)
    dist = [float(1e9)]*(n)
    visit = [0]*n
    parent = list(range(n)) #to store path
    dist[start] = 0 + heuristic(start, goal)

    while frontier:
        dis, city = heapq.heappop(frontier)
        if city == goal:
            path = []
            city = goal
            while parent[city] != city:
                path.append(index_city_mapping[city])
                city = parent[city]
            path.append(index_city_mapping[start])
            path.reverse()
            return path, dist[goal]
        for adj_city, road_distance in adj[city]:
            if dis-heuristic(city, goal) + road_distance +  heuristic(adj_city, goal) < dist[adj_city]:
                heapq.heappush(frontier, (dis-heuristic(city, goal) + road_distance +  heuristic(adj_city, goal), adj_city))
                parent[adj_city] = city
                dist[adj_city] = dis - heuristic(city, goal) + road_distance + heuristic(adj_city, goal)


In [10]:
i_heuristic(city_index_mapping['Hubli'], 'a')

2101.0

In [13]:
print(uniform_cost(len(city_index_mapping), city_index_mapping['Surat'], city_index_mapping['Agra'], adj))


(['Surat', 'Ahmedabad', 'Agra'], 1141.0)


In [14]:
print(astar_search(len(city_index_mapping), city_index_mapping['Surat'], city_index_mapping['Agra'], adj, i_heuristic))

(['Surat', 'Indore', 'Agra'], 3476.0)


In [20]:
# Ludhiana Coimbatore
# Surat Agra
while True:
    start = input("Enter intitial destination : ")
    goal = input("Enter final destination : ")
    print("Uniform cost search: ")
    print(uniform_cost(len(city_index_mapping), city_index_mapping[start], city_index_mapping[goal], adj))
    print("A* search with admissible heuristic: ")
    print(astar_search(len(city_index_mapping), city_index_mapping[start], city_index_mapping[goal], adj, a_heuristic))
    print("A* search with inadmissible heuristic: ")
    print(astar_search(len(city_index_mapping), city_index_mapping[start], city_index_mapping[goal], adj, i_heuristic))

    

Uniform cost search: 
(['Ludhiana', 'Chandigarh', 'Cochin', 'Coimbatore'], 2265.0)
A* search with admissible heuristic: 
(['Ludhiana'], 0.0)
A* search with inadmissible heuristic: 
(['Ludhiana'], 3027.0)
Uniform cost search: 
(['Surat', 'Ahmedabad', 'Agra'], 1141.0)
A* search with admissible heuristic: 
(['Surat'], 0.0)
A* search with inadmissible heuristic: 
(['Surat'], 1912.0)


Two origin-destination pairs for which the inadmissible heuristic expands fewer nodes are as follows:
    1. Ludhiana and Coimbatore
        The optimal path between them is ['Ludhiana', 'Chandigarh', 'Cochin', 'Coimbatore'] whereas the path given by inadmissible heuristic is ['Ludhiana', 'Jaipur', 'Indore', 'Nasik', 'Coimbatore']
    2. Surat and Agra
        The optimal path between them is ['Surat', 'Ahmedabad', 'Agra'] whereas the path given by inadmissible heuristic is ['Surat', 'Indore', 'Agra']